# Tarea 1 AS4501-1

Profesor de catedra: Francisco Förster

Autor: Álvaro Cáceres

## Problem 1
proposito blablabla

The file `fundamental-plane.csv` precisely contains data about these parameters originally compiled by Djorgovski & Davis in 1987. Moving forward in this problem we will be working with this data.